In [ ]:
import cv2
import matplotlib.pyplot as plt
from binary import img_to_binary

In [ ]:
plt.rcParams["axes.labelsize"] = 25
plt.rcParams["axes.titlesize"] = 20
plt.rcParams["axes.linewidth"] = 1
plt.rcParams["axes.titlepad"] = 25
plt.rcParams["axes.labelpad"] = 10
plt.rc("xtick.major", width=2, size=5, pad=10)
plt.rc("ytick.major", width=2, size=5, pad=10)
plt.rc("xtick", labelsize=15, direction="out")
plt.rc("ytick", labelsize=15, direction="out")

In [ ]:
input_filepath = "../../../data/POM/imgs/crack.jpeg"

In [ ]:
binary_img = img_to_binary(input_filepath, 90)

In [ ]:
# クラック画像を二値化表示
def crack_binary_img(binary_data):
    fig, ax = plt.subplots()

    ax.imshow(binary_data, cmap="gray")
    ax.set_title("crack binary image")
    ax.set_xlabel("$X$")
    ax.set_ylabel("$Y$")
    ax.set_xlim(0, binary_img.shape[1])
    ax.set_ylim(binary_img.shape[0], 0)

    return binary_data


crack_binary_img(binary_img)

In [ ]:
# クラック画像のエッジ検出
def crack_img_edge_detect(binary_data):
    edge_detect_img = cv2.Canny(binary_data, threshold1=100, threshold2=255)
    fig, ax = plt.subplots()

    ax.imshow(edge_detect_img, cmap="gray")
    ax.set_title("crack edge detection image")
    ax.set_xlabel("$X$")
    ax.set_ylabel("$Y$")
    ax.set_xlim(0, binary_img.shape[1])
    ax.set_ylim(binary_img.shape[0], 0)

    return edge_detect_img


crack_img_edge_detect(binary_img)

In [ ]:
# クラック画像のエッジの輪郭を抽出
def edge_count_contours(binary_data):
    edge_detect_img = crack_img_edge_detect(binary_data)

    contours, _ = cv2.findContours(
        edge_detect_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    count_contours = cv2.drawContours(binary_img, contours, -1, (0, 255, 0), 2)

    return contours, count_contours


edge_count_contours(binary_img)

In [ ]:
# 抽出した輪郭数を表示
edge_detect_img = cv2.Canny(binary_img, 100, 255)
contours, hierachy = cv2.findContours(
    edge_detect_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

print("contours :", len(contours))

In [ ]:
# 輪郭を閾値でわける
filtered_contours = [cnt for cnt in contours if len(cnt) > 35]

print("filtered_contours :", len(filtered_contours))

In [ ]:
# 画像の足し合わせ
fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(8, 8))

img = edge_detect_img

edge_list = []
for i in range(len(filtered_contours)):
    img = cv2.drawContours(
        cv2.cvtColor(edge_detect_img, cv2.COLOR_GRAY2RGB),
        filtered_contours,  # 輪郭を保存したリスト
        i,  # リストの何番目か
        (255, 255, 255),  # 白
        10,  # 線の太さ
    )

    edge_list.append(img)

img = edge_list[0]
for i in range(len(edge_list) - 1):
    img = cv2.add(img, edge_list[i + 1])

ax.imshow(img)
ax.set_title("added edge detect image contour{}".format(i))
ax.set_xlabel("$X$")
ax.set_ylabel("$Y$")
ax.set_xlim(0, binary_img.shape[1])
ax.set_ylim(binary_img.shape[0], 0)

plt.tight_layout()
plt.show()

In [ ]:
edge = cv2.Canny(img, 100, 255)

fig, ax = plt.subplots(1, 2, figsize=(20, 20), sharex=True, sharey=True)
ax[0].imshow(img, cmap="gray")
ax[0].set_title("orginal")
ax[1].imshow(edge, cmap="gray")
ax[1].set_title("edged")
ax[0].set_xticks([])
ax[0].set_yticks([])
plt.show()

# やったこと

画像を二値化

Canny法でエッジを検出

輪郭の抽出

# 結果

1つのクラックに対して輪郭を複数検出した

# 原因と考えられるもの

クラックの画像が線として認識されなかった

クラック画像が複雑である



# 予想される解決策



# クラック画像が連続的な線であるかどうか

エッジ検出画像を拡大した結果、クラックのエッジが白っぽくなっていたため、クラックは連続的に検出されていたが白黒だけでなくグレーでエッジが出力されていた

明るくハイライトされたクラックのエッジ部分の暗い部分でクラックの線の輪郭が途切れたと考えられる

# 問題の解決策

エッジ検出画像の暗い部分を明るく表示するために二値化画像に変換してみる